**Imports**

In [252]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from typing import Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import math
import QuantLib as ql

**Define price fetcher**

In [253]:
def get_spot_price(ticker):
    """
    Fetches the current spot price for a stock ticker. Falls back to the most recent
    close if a live price is not available.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').

    Returns
    -------
    float or None
        Spot price (live if available, else last close). Returns None if unavailable.
    """

    try:
        tk = yf.Ticker(ticker)
        
        # Attempt to fetch live price
        live_price = tk.fast_info.get("last_price", None)
        if live_price and live_price > 0:
            return live_price

        # Fallback: most recent close
        hist = tk.history(period="1d")
        if not hist.empty:
            fallback_price = hist["Close"].iloc[-1]
            print(f"[{ticker}] Live price unavailable — using last close: {fallback_price:.2f}")
            return fallback_price

        print(f"[{ticker}] No live or historical data available.")
        return None

    except Exception as e:
        print(f"[{ticker}] Spot price fetch failed: {e}")
        return None

**Define Option Chain Fetcher**

In [254]:
def get_option_chains_all(ticker: str,
                                  max_workers: int = 8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Fetches option chains (calls and puts) for every available expiry of a given ticker,
    performing API requests in parallel to reduce total fetch time.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').
    max_workers : int, optional
        Maximum number of threads to use for concurrent fetching (default is 8).

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        - calls_df: DataFrame containing all calls across expiries, with added columns:
            * 'option_type' = 'call'
            * 'expiration'  = expiry date string 'YYYY-MM-DD'
            * 'TTM'         = time to maturity in years
        - puts_df: DataFrame containing all puts with the same added columns.
    """
    stock = yf.Ticker(ticker)
    expiries = stock.options  # list of expiry date strings
    today = datetime.now().date()

    calls_accum = []
    puts_accum  = []

    def fetch_chain(expiry: str):
        """Fetch calls/puts for a single expiry and return (expiry, calls_df, puts_df)."""
        try:
            chain = stock.option_chain(expiry)
            calls = chain.calls.copy()
            puts  = chain.puts.copy()
        except Exception as e:
            # Return None on error so we can skip later
            return expiry, None, None

        # Tag each row with type and expiration
        calls['option_type']  = 'call'
        puts ['option_type']  = 'put'
        calls['expiration']   = expiry
        puts ['expiration']   = expiry

        # Compute time-to-maturity once
        exp_date = datetime.strptime(expiry, "%Y-%m-%d").date()
        ttm = max((exp_date - today).days / 365.0, 0.0)
        calls['TTM'] = ttm
        puts ['TTM'] = ttm

        return expiry, calls, puts

    # Fetch in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_chain, exp) for exp in expiries]
        for future in as_completed(futures):
            expiry, calls_df, puts_df = future.result()
            if calls_df is not None and not calls_df.empty:
                calls_accum.append(calls_df)
            if puts_df  is not None and not puts_df.empty:
                puts_accum.append(puts_df)

    # Concatenate results
    all_calls = pd.concat(calls_accum, ignore_index=True) if calls_accum else pd.DataFrame()
    all_puts  = pd.concat(puts_accum,  ignore_index=True) if puts_accum  else pd.DataFrame()

    # Fetch dividend yield for the company
    dividendYield = stock.info.get("dividendYield")/100 # percentages on decimal basis
    all_calls["dividendYield"] = dividendYield
    all_puts["dividendYield"] = dividendYield

    all_calls["ticker"] = ticker
    all_puts["ticker"] = ticker

    spot_price = get_spot_price(ticker)
    all_calls["spot_price"] = spot_price
    all_puts["spot_price"] = spot_price

    return all_calls, all_puts


Get option chain and dividend yield

In [255]:
calls, puts = get_option_chains_all("AAPL")

[AAPL] Live price unavailable — using last close: 268.45


In [256]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251205C00165000,2025-10-31 13:30:12+00:00,165.0,111.70,100.80,104.35,0.00,0.000000,1.0,1,0.978516,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
1,AAPL251205C00170000,2025-10-31 14:11:10+00:00,170.0,100.24,96.05,98.50,0.00,0.000000,1.0,1,0.615238,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
2,AAPL251205C00185000,2025-10-31 13:30:12+00:00,185.0,92.00,81.00,84.50,0.00,0.000000,10.0,10,0.802980,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
3,AAPL251205C00190000,2025-11-06 18:19:57+00:00,190.0,81.70,76.20,79.30,0.00,0.000000,2.0,29,0.723391,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
4,AAPL251205C00200000,2025-11-05 17:02:02+00:00,200.0,70.70,66.10,69.20,0.00,0.000000,5.0,6,0.616947,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,0.276954,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1222,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,0.277015,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1223,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,0.276374,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1224,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,0.276649,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012


In [257]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.0,0.0,1.0,14,0.734378,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.0,0.0,1.0,0,0.632816,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.0,0.0,1.0,6,0.621098,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.0,0.0,2.0,2,0.566411,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.0,0.0,1.0,1,0.539067,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,AAPL280121P00380000,2025-10-31 18:34:12+00:00,380.0,108.95,112.90,113.50,0.0,0.0,5.0,8,0.153756,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1053,AAPL280121P00390000,2025-11-05 15:56:30+00:00,390.0,121.25,122.50,123.10,0.0,0.0,2.0,24,0.154885,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1054,AAPL280121P00400000,2025-10-30 16:43:32+00:00,400.0,128.50,132.50,133.10,0.0,0.0,NaN,0,0.162850,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1055,AAPL280121P00450000,2025-10-29 13:35:18+00:00,450.0,180.74,180.50,185.00,0.0,0.0,NaN,0,0.232857,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012


In [258]:
calls.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1204.000000,1226.000000,1226.000000,1226.000000,1.226000e+03,1226.000000
mean,241.272431,66.715783,65.478083,66.689576,-0.670294,-6.752271,479.211794,2808.033442,0.560822,0.672525,3.900000e-03,268.450012
std,115.648359,70.227935,69.984938,71.235853,3.231657,14.039237,3856.273992,7073.409427,0.721523,0.695275,4.338578e-19,0.000000
min,5.000000,0.010000,0.000000,0.000000,-8.389999,-99.371070,1.000000,0.000000,0.000010,0.000000,3.900000e-03,268.450012
25%,155.000000,2.915000,2.515000,2.605000,-1.707500,-8.279793,2.000000,65.000000,0.274238,0.095890,3.900000e-03,268.450012
50%,240.000000,40.450000,38.450000,39.350000,-0.060000,-1.518472,11.000000,379.000000,0.343757,0.441096,3.900000e-03,268.450012
75%,320.000000,118.677500,116.125000,117.975000,0.000000,0.000000,68.250000,2013.500000,0.553273,1.112329,3.900000e-03,268.450012
max,530.000000,266.040000,260.850000,264.500000,70.640000,100.000000,89573.000000,88993.000000,7.632813,2.205479,3.900000e-03,268.450012


Initial Dataset Clean

In [259]:
def initial_dataset_clean(dataset : pd.DataFrame):
    # Removing columns that will not be used for training or filtering
    #cols_to_drop = ["contractSymbol", "lastTradeDate", "change", "percentChange", "expiration", "inTheMoney"]
    #dataset.drop(columns=[c for c in cols_to_drop if c in dataset.columns], inplace=True)

    # Removing NA values for stale options
    dataset.dropna(axis=0,subset=["volume"], inplace=True)

    
    return dataset

In [260]:
calls_clean = initial_dataset_clean(calls)
calls_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251205C00165000,2025-10-31 13:30:12+00:00,165.0,111.70,100.80,104.35,0.00,0.000000,1.0,1,0.978516,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
1,AAPL251205C00170000,2025-10-31 14:11:10+00:00,170.0,100.24,96.05,98.50,0.00,0.000000,1.0,1,0.615238,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
2,AAPL251205C00185000,2025-10-31 13:30:12+00:00,185.0,92.00,81.00,84.50,0.00,0.000000,10.0,10,0.802980,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
3,AAPL251205C00190000,2025-11-06 18:19:57+00:00,190.0,81.70,76.20,79.30,0.00,0.000000,2.0,29,0.723391,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
4,AAPL251205C00200000,2025-11-05 17:02:02+00:00,200.0,70.70,66.10,69.20,0.00,0.000000,5.0,6,0.616947,True,REGULAR,USD,call,2025-12-05,0.076712,0.0039,AAPL,268.450012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,0.276954,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1222,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,0.277015,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1223,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,0.276374,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
1224,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,0.276649,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012


In [261]:
calls_clean.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1.204000e+03,1204.000000
mean,240.901163,66.652924,65.563679,66.773140,-0.682542,-6.875652,479.211794,2858.700166,0.558275,0.676867,3.900000e-03,268.450012
std,114.663472,70.007036,69.687471,70.933147,3.259790,14.137031,3856.273992,7127.748409,0.723745,0.697032,4.338611e-19,0.000000
min,5.000000,0.010000,0.000000,0.000000,-8.389999,-99.371070,1.000000,0.000000,0.000010,0.000000,3.900000e-03,268.450012
25%,155.000000,3.050000,2.902500,2.952500,-1.727501,-8.382243,2.000000,72.000000,0.273948,0.095890,3.900000e-03,268.450012
50%,240.000000,40.450000,38.925000,39.525000,-0.090000,-1.738211,11.000000,397.500000,0.343017,0.441096,3.900000e-03,268.450012
75%,320.000000,118.415000,116.275000,118.237500,0.000000,0.000000,68.250000,2058.000000,0.546361,1.112329,3.900000e-03,268.450012
max,530.000000,266.040000,260.850000,264.500000,70.640000,100.000000,89573.000000,88993.000000,7.632813,2.205479,3.900000e-03,268.450012


In [262]:
puts_clean = initial_dataset_clean(puts)
puts_clean

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.0,0.0,1.0,14,0.734378,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.0,0.0,1.0,0,0.632816,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.0,0.0,1.0,6,0.621098,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.0,0.0,2.0,2,0.566411,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.0,0.0,1.0,1,0.539067,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,AAPL280121P00340000,2025-11-06 20:55:40+00:00,340.0,77.62,78.90,79.55,0.0,0.0,5.0,11,0.177849,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1049,AAPL280121P00350000,2025-11-06 19:29:53+00:00,350.0,83.55,86.70,87.45,0.0,0.0,3.0,114,0.171334,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1050,AAPL280121P00360000,2025-11-04 17:50:40+00:00,360.0,92.33,94.95,95.65,0.0,0.0,2.0,9,0.163750,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012
1052,AAPL280121P00380000,2025-10-31 18:34:12+00:00,380.0,108.95,112.90,113.50,0.0,0.0,5.0,8,0.153756,True,REGULAR,USD,put,2028-01-21,2.205479,0.0039,AAPL,268.450012


**Join Calls and Puts**

In [263]:
df = pd.concat([puts_clean, calls_clean], ignore_index=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,0.734378,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,0.632816,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,0.621098,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,0.566411,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,0.539067,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,0.276954,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,0.277015,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,0.276374,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,0.276649,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012


In [264]:
def optionType(OPTtype):
    # Puts are type 1
    if OPTtype == "put":
        return "put"
    else:
    # Calls are type 0
        return "call"

In [265]:
df["optionType"] = df["option_type"].apply(optionType)
#df.drop(columns="option_type", inplace=True)
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,...,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,...,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,...,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,...,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,...,False,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,...,False,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call


**Interest Rate Interpolation**

In [266]:
# Needs to figure out if we're using a interpolation structure to match each of them, also considering zero bond rates for the interpolation
def interest_rate(row):
    # Need to change this to interpolation calculation
    return (0.04)

In [267]:
df["r"] = .04
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,...,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,...,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,...,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,...,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,...,REGULAR,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,...,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,...,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,...,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,...,REGULAR,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04


**Forward log-Moneyness**

We decided to use forward log-moneyness because its adjusted for r and q which are crucial in pricing the EEP

In [268]:

def forward_log_moneyness(S: float, r:float, q:float, T:float, K:float):
    """
    Calculates the log_moneyness in relation to the Forward Price of the stock

    Parameters
    ----------
    S : float
        Current spot_price of the stock.
    r : float
        Risk-free rate associated with that particular option maturity
    q : float
        Dividend-yield rate associated with that particular option maturity
    T : float
        Maturity of the option contract
    K : float
        Strike price of the option contract

    Returns
    -------
    Float
        - Forward log moneyness 
    """
    forward_price = S * np.exp((r - q)*T)
    
    return np.log(K/forward_price)

In [269]:
df["forward_log_moneyness"] = forward_log_moneyness(
    S=df["spot_price"].to_numpy(),
    r=df["r"].to_numpy(),
    q=df["dividendYield"].to_numpy(),
    T=df["TTM"].to_numpy(),
    K=df["strike"].to_numpy(),
)

df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,...,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.653792
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,...,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.520260
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,...,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.489489
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,...,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.430648
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,...,USD,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.402477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,...,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.522123
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,...,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.542326
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,...,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.562128
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,...,USD,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.581546


Calculate Mid Price

In [270]:
def midPrice(bid: float, ask: float):
    """
    Calcualtes the mid-price of the option based on bid and ask prices

    Parameters
    ----------
    bid : float
        Current bid price of the option
    ask : float
        Current ask price of the option

    Returns
    -------
    Float
        - Mid price
    """

    return ((bid+ask)/2)
    

In [271]:
df["midPrice"] = midPrice(df["bid"], df["ask"])
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,option_type,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,...,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.653792,0.010
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,...,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.520260,0.020
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,...,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.489489,0.030
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,...,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.430648,0.040
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,...,put,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.402477,0.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,...,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.522123,4.600
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,...,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.542326,4.150
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,...,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.562128,3.700
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,...,call,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.581546,3.350


In [272]:


# =========================
# Debug helper
# =========================
def _d(msg, **k):
    items = " | ".join(f"{kk}={vv}" for kk, vv in k.items())
    print(f"[deAm] {msg}" + (f" :: {items}" if items else ""))

def _get_mid(row):
    """Return mid price from either 'midPrice' or 'mid_price' if present, else None."""
    v = row.get('midPrice', row.get('mid_price', None))
    try:
        return float(v) if v is not None else None
    except Exception:
        return None

# =========================
# Curves & maturity
# =========================
def _setup_ts(eval_date, r, q):
    """
    Flat, continuously compounded r and q term structures.
    """
    dc = ql.Actual365Fixed()
    r_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(eval_date, ql.QuoteHandle(ql.SimpleQuote(float(r))),
                       dc, ql.Continuous, ql.NoFrequency))
    q_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(eval_date, ql.QuoteHandle(ql.SimpleQuote(float(q))),
                       dc, ql.Continuous, ql.NoFrequency))
    return dc, r_ts, q_ts

def _to_maturity(eval_date, T_years):
    """
    Convert year fraction to Date using Actual/365, with ≥ 1 day.
    """
    days = max(1, int(round(float(T_years) * 365.0)))
    return eval_date + days

# =========================
# Core: de-Americanize via binomial inversion (robust)
# =========================
def _deam_one_tree(row, eval_date, steps, tree, sigma_floor=1e-3):
    """
    Attempt de-Americanization using a single tree flavor. Returns (price or None, errstr or None).
    """
    S = float(row['spot_price'])
    K = float(row['strike'])
    T = float(row['TTM'])
    r = float(row['r'])
    q = float(row['dividendYield'])
    P = _get_mid(row)
    opt_is_call = str(row.get('optionType', row.get('option_type'))).lower() == 'call'

    # ---------- input guards ----------
    if P is None or T <= 0 or not np.isfinite(P) or P <= 0 or S <= 0 or K <= 0:
        return None, "bad inputs"
    if q > 1.0:  # defensive if someone passed percent
        q = q / 100.0
        _d("q looked like percent; converted to decimal", q=q)

    # ---------- no-arb bounds ----------
    df_r = np.exp(-r*T); df_q = np.exp(-q*T)
    if opt_is_call:
        lb, ub = max(0.0, S*df_q - K*df_r), S*df_q
    else:
        lb, ub = max(0.0, K*df_r - S*df_q), K*df_r
    if not (lb - 1e-8 <= P <= ub + 1e-8):
        return None, "no-arb violation"

    # ---------- trivial case: American call with ~zero dividend ----------
    if opt_is_call and abs(q) <= 1e-4:
        return float(P), None

    # ---------- engine & instruments ----------
    ql.Settings.instance().evaluationDate = eval_date
    dc, r_ts, q_ts = _setup_ts(eval_date, r, q)
    spot = ql.QuoteHandle(ql.SimpleQuote(S))

    # keep SimpleQuote so we can mutate σ during solve
    vol_sq = ql.SimpleQuote(max(0.30, sigma_floor))
    vol_h  = ql.QuoteHandle(vol_sq)
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, ql.NullCalendar(), vol_h, dc))
    process = ql.BlackScholesMertonProcess(spot, q_ts, r_ts, vol_ts)

    maturity = _to_maturity(eval_date, T)
    ql_type = ql.Option.Call if opt_is_call else ql.Option.Put
    am_opt = ql.VanillaOption(ql.PlainVanillaPayoff(ql_type, K), ql.AmericanExercise(eval_date, maturity))
    eu_opt = ql.VanillaOption(ql.PlainVanillaPayoff(ql_type, K), ql.EuropeanExercise(maturity))

    # Tree-specific adjustments
    t = tree.lower()
    use_steps = int(steps)
    if t == "lr" and use_steps % 2 == 0:
        use_steps += 1  # LR needs odd
    try:
        am_opt.setPricingEngine(ql.BinomialVanillaEngine(process, t, use_steps))
    except Exception as e:
        return None, f"engine init failed: {e}"

    # ---------- target function f(σ) with exception safety ----------
    class _Res:
        def __call__(self, sigma):
            s = max(float(sigma), sigma_floor)
            try:
                vol_sq.setValue(s)
                val = am_opt.NPV()  # can throw "negative probability" if p ∉ [0,1]
                if not np.isfinite(val):
                    _d("am NPV not finite", sigma=s, steps=use_steps, tree=t)
                    return np.nan
                return val - P
            except Exception as e:
                _d("am NPV exception", tree=t, steps=use_steps, sigma=s, err=str(e))
                return np.nan  # signal to bracket-expander to adjust

    f = _Res()

    # ---------- bracket with robust expansion ----------
    lo, hi = max(sigma_floor, 1e-3), 6.0
    f_lo, f_hi = f(lo), f(hi)
    _d("initial bracket", tree=t, steps=use_steps, lo=lo, f_lo=f_lo, hi=hi, f_hi=f_hi, P=P)
    expands = 0
    while (not np.isfinite(f_lo) or not np.isfinite(f_hi) or f_lo * f_hi > 0) and expands < 12:
        lo *= 0.6
        hi *= 1.7
        lo = max(lo, sigma_floor)
        f_lo, f_hi = f(lo), f(hi)
        expands += 1
        _d("expand bracket", tree=t, expands=expands, lo=lo, f_lo=f_lo, hi=hi, f_hi=f_hi)
    if not np.isfinite(f_lo) or not np.isfinite(f_hi) or f_lo * f_hi > 0:
        return None, "no sigma bracket"

    # ---------- root solve ----------
    try:
        sigma_star = float(ql.Brent().solve(f, 1e-8, max(0.30, lo*1.5), lo, hi))
        _d("sigma*", tree=t, sigma_star=sigma_star)
    except Exception as e:
        return None, f"Brent failed: {e}"

    # ---------- European analytic price ----------
    try:
        vol_sq.setValue(max(sigma_star, sigma_floor))
        eu_opt.setPricingEngine(ql.AnalyticEuropeanEngine(process))
        p_eu = float(eu_opt.NPV())
        _d("eu price", tree=t, p_eu=p_eu)
        return p_eu, None
    except Exception as e:
        return None, f"eu analytic failed: {e}"

def _deamericanize_price_binomial(row, eval_date=None, steps=400, tree="jr"):
    """
    Convert an American mid price to a European-equivalent price.
    Tries a sequence of trees for robustness if the chosen one fails.
    """
    if eval_date is None:
        eval_date = ql.Date.todaysDate()

    # First try requested tree, then robust fallbacks
    tree_try = [tree.lower()]
    for t in ["jr", "trigeorgis", "tian", "crr", "lr"]:
        if t not in tree_try:
            tree_try.append(t)

    for t in tree_try:
        p, err = _deam_one_tree(row, eval_date, steps, t, sigma_floor=1e-3)
        if p is not None and np.isfinite(p) and p > 0:
            return p
        _d("tree attempt failed", tree=t, err=err)
    return None

# =========================
# BS IV from price (robust)
# =========================
def _row_bs_iv_from_price(row, eval_date=None, iv_guess=0.25, use_deam=True):
    """
    Compute BS implied vol. If use_deam, first de-Americanize to a European price.
    Returns float IV or None.
    """
    S = float(row['spot_price']); K = float(row['strike']); T = float(row['TTM'])
    r = float(row['r']); q = float(row['dividendYield'])
    P_raw = _get_mid(row)
    opt_is_call = str(row.get('optionType', row.get('option_type'))).lower() == 'call'

    if P_raw is None or T <= 0 or S <= 0 or K <= 0:
        _d("IV: bad inputs", S=S, K=K, T=T, r=r, q=q, P=P_raw)
        return None

    if eval_date is None:
        eval_date = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = eval_date

    price_for_iv = P_raw
    if use_deam:
        P_eu = _deamericanize_price_binomial(row, eval_date=eval_date)
        if P_eu is None or not np.isfinite(P_eu) or P_eu <= 0:
            _d("IV: deAm failed", P_raw=P_raw, P_eu=P_eu)
            return None
        price_for_iv = float(P_eu)

    dc, r_ts, q_ts = _setup_ts(eval_date, r, q)
    cal = ql.NullCalendar()
    maturity = _to_maturity(eval_date, T)
    spot = ql.QuoteHandle(ql.SimpleQuote(S))
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, cal, iv_guess, dc))
    proc = ql.BlackScholesMertonProcess(spot, q_ts, r_ts, vol_ts)
    ql_type = ql.Option.Call if opt_is_call else ql.Option.Put
    opt = ql.VanillaOption(ql.PlainVanillaPayoff(ql_type, K), ql.EuropeanExercise(maturity))

    # Try QuantLib's built-in solver with wide bounds
    try:
        iv = opt.impliedVolatility(price_for_iv, proc, 1e-6, 2000, 1e-9, 12.0)
        iv = float(iv) if np.isfinite(iv) and iv > 0 else None
        _d("IV: QL solver", iv=iv, price_for_iv=price_for_iv)
        if iv is not None:
            return iv
    except Exception as e:
        _d("IV: QL solver exception", err=str(e), price_for_iv=price_for_iv)

    # Fallback: bisection using analytic engine (monotone in vol)
    try:
        lo, hi = 1e-9, 12.0
        for it in range(120):
            mid = 0.5 * (lo + hi)
            vts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, cal, mid, dc))
            p = ql.BlackScholesMertonProcess(spot, q_ts, r_ts, vts)
            opt.setPricingEngine(ql.AnalyticEuropeanEngine(p))
            p_mid = opt.NPV()
            if not np.isfinite(p_mid):
                _d("IV: fallback p_mid not finite", it=it, mid=mid)
                return None
            if abs(p_mid - price_for_iv) < 1e-8:
                _d("IV: fallback converged", it=it, mid=mid)
                return mid
            if p_mid < price_for_iv:
                lo = mid
            else:
                hi = mid
        _d("IV: fallback no converge", lo=lo, hi=hi)
        return None
    except Exception as e:
        _d("IV: fallback exception", err=str(e))
        return None

# =========================
# Heston calibration & pricing
# =========================
def _calibrate_heston(group_df: pd.DataFrame, eval_date=None, init=None):
    """
    Calibrate a Heston model to one group (same instrument/day) using IVs from mid prices.
    Requires ≥ ~5 valid helpers across strikes/maturities.
    """
    if eval_date is None:
        eval_date = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = eval_date

    S = float(group_df['spot_price'].iloc[0])
    r = float(group_df['r'].iloc[0])
    q = float(group_df['dividendYield'].iloc[0])

    _, r_ts, q_ts = _setup_ts(eval_date, r, q)
    cal = ql.NullCalendar()
    spot_h = ql.QuoteHandle(ql.SimpleQuote(S))

    helpers = []
    for idx, row in group_df.iterrows():
        iv = _row_bs_iv_from_price(row, eval_date=eval_date, use_deam=True)
        _d("helper IV", idx=idx, iv=iv)
        if iv is None:
            continue
        K = float(row['strike']); T = float(row['TTM'])
        tenor = ql.Period((_to_maturity(eval_date, T) - eval_date), ql.Days)
        helpers.append(ql.HestonModelHelper(tenor, cal,S, K,
                                            ql.QuoteHandle(ql.SimpleQuote(iv)),r_ts,q_ts)
)

    _d("helpers count", n=len(helpers))
    if len(helpers) < 5:
        raise ValueError("Not enough valid options to calibrate Heston (need ≥ ~5 across strikes/maturities).")

    # Initial params (can be overridden with init)
    p = dict(v0=0.04, kappa=1.5, theta=0.04, sigma=0.3, rho=-0.7)
    if init: p.update(init)

    process = ql.HestonProcess(r_ts, q_ts, spot_h, p['v0'], p['kappa'], p['theta'], p['sigma'], p['rho'])
    model = ql.HestonModel(process)
    engine = ql.AnalyticHestonEngine(model)
    for h in helpers:
        h.setPricingEngine(engine)

    om = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
    endc = ql.EndCriteria(500, 50, 1e-8, 1e-8, 1e-8)
    model.calibrate(helpers, om, endc)

    # Dump fitted params for visibility
    try:
        v0, kappa, theta, sigma, rho = model.params()
        _d("Heston calibrated", v0=v0, kappa=kappa, theta=theta, sigma=sigma, rho=rho)
    except Exception:
        pass
    return model

def _price_eu_heston(row, model: ql.HestonModel, eval_date=None):
    """
    Price a single European option under calibrated Heston.
    """
    if eval_date is None:
        eval_date = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = eval_date

    K = float(row['strike'])
    T = float(row['TTM'])
    maturity = _to_maturity(eval_date, T)
    ql_type = ql.Option.Call if str(row.get('optionType', row.get('option_type'))).lower() == 'call' else ql.Option.Put

    opt = ql.VanillaOption(ql.PlainVanillaPayoff(ql_type, K), ql.EuropeanExercise(maturity))
    opt.setPricingEngine(ql.AnalyticHestonEngine(model))
    p = float(opt.NPV())
    _d("Heston EU price", K=K, T=T, p=p)
    return p

# =========================
# Public API
# =========================
def calibrate_and_price_heston_european(df: pd.DataFrame,
                                        group_cols=('ticker',),
                                        eval_date: ql.Date | None = None,
                                        init: dict | None = None) -> pd.Series:
    """
    Calibrate Heston per group from mid prices, then return ONLY the European-equivalent
    price for each row as 'V_EU_Heston'.
    """
    if eval_date is None:
        eval_date = ql.Date.todaysDate()

    results = pd.Series(index=df.index, dtype=float, name='V_EU_Heston')

    # Attach index to help correlate debug lines to inputs
    df = df.copy()
    df['idx'] = df.index

    for gkey, grp in df.groupby(list(group_cols), dropna=False):
        g = grp.copy()
        _d("=== group start ===", group=gkey, rows=len(g))
        try:
            model = _calibrate_heston(g, eval_date=eval_date, init=init)
        except Exception as e:
            _d("group calibration FAILED", group=gkey, err=str(e))
            continue

        prices = g.apply(lambda r: _price_eu_heston(r, model, eval_date), axis=1)
        results.loc[g.index] = prices.values
        _d("group priced", group=gkey)

    return results


In [273]:

V_eu = calibrate_and_price_heston_european(df, group_cols=("ticker", ))
df = df.join(V_eu)
df

[deAm] === group start === :: group=('AAPL',) | rows=2231
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-0.01 | hi=6.0 | f_hi=63.80411442899315 | P=0.01


[deAm] sigma* :: tree=jr | sigma_star=0.7578521652106623
[deAm] eu price :: tree=jr | p_eu=0.01024206702687044
[deAm] IV: QL solver :: iv=0.7578521643836771 | price_for_iv=0.01024206702687044
[deAm] helper IV :: idx=0 | iv=0.7578521643836771
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-0.02 | hi=6.0 | f_hi=77.3969249696465 | P=0.02
[deAm] sigma* :: tree=jr | sigma_star=0.6488294647775062
[deAm] eu price :: tree=jr | p_eu=0.020414608365889424
[deAm] IV: QL solver :: iv=0.6488292672541204 | price_for_iv=0.020414608365889424
[deAm] helper IV :: idx=1 | iv=0.6488292672541204
[deAm] initial bracket :: tree=jr | steps=400 | lo=0.001 | f_lo=-0.03 | hi=6.0 | f_hi=80.92193096758939 | P=0.03
[deAm] sigma* :: tree=jr | sigma_star=0.6365817140461942
[deAm] eu price :: tree=jr | p_eu=0.030427951273926914
[deAm] IV: QL solver :: iv=0.6365816846440295 | price_for_iv=0.030427951273926914
[deAm] helper IV :: idx=2 | iv=0.6365816846440295
[deAm] initial bracket :: tree=jr | steps=400

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,expiration,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness,midPrice,V_EU_Heston
0,AAPL251205P00140000,2025-11-06 18:53:03+00:00,140.0,0.01,0.00,0.02,0.00,0.000000,1.0,14,...,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.653792,0.010,0.000000e+00
1,AAPL251205P00160000,2025-11-04 14:35:00+00:00,160.0,0.03,0.01,0.03,0.00,0.000000,1.0,0,...,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.520260,0.020,1.275058e-13
2,AAPL251205P00165000,2025-11-05 20:24:52+00:00,165.0,0.02,0.02,0.04,0.00,0.000000,1.0,6,...,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.489489,0.030,1.374230e-12
3,AAPL251205P00175000,2025-11-04 20:57:14+00:00,175.0,0.04,0.03,0.05,0.00,0.000000,2.0,2,...,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.430648,0.040,1.153786e-10
4,AAPL251205P00180000,2025-11-06 15:20:53+00:00,180.0,0.03,0.03,0.06,0.00,0.000000,1.0,1,...,2025-12-05,0.076712,0.0039,AAPL,268.450012,put,0.04,-0.402477,0.045,9.257916e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,AAPL280121C00490000,2025-10-31 19:49:20+00:00,490.0,4.90,4.50,4.70,0.00,0.000000,1.0,25,...,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.522123,4.600,6.507557e+00
2227,AAPL280121C00500000,2025-11-07 18:21:15+00:00,500.0,4.35,4.05,4.25,0.00,0.000000,7.0,285,...,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.542326,4.150,5.931664e+00
2228,AAPL280121C00510000,2025-11-06 16:40:48+00:00,510.0,4.05,3.60,3.80,0.00,0.000000,10.0,54,...,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.562128,3.700,5.408409e+00
2229,AAPL280121C00520000,2025-11-07 15:34:36+00:00,520.0,3.69,3.25,3.45,0.11,3.072630,1.0,14,...,2028-01-21,2.205479,0.0039,AAPL,268.450012,call,0.04,0.581546,3.350,4.932922e+00


In [274]:
df.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price,r,forward_log_moneyness,midPrice,V_EU_Heston
count,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.000000,2231.0000,2231.000000,2231.00,2231.000000,2231.000000,2.231000e+03
mean,224.989915,44.579121,43.601147,44.436867,-0.228391,-1.278455,444.853429,2598.144330,0.551284,0.670727,0.0039,268.450012,0.04,-0.373159,44.019007,4.398766e+01
std,107.163084,61.775048,60.870248,61.977807,2.487978,16.849884,3616.518884,6261.611005,0.652002,0.690688,0.0000,0.000000,0.00,0.705515,61.423369,6.251795e+01
min,5.000000,0.010000,0.000000,0.000000,-8.389999,-99.371070,1.000000,0.000000,0.000010,0.000000,0.0039,268.450012,0.04,-4.062845,0.000000,-4.727788e-11
25%,145.000000,0.445000,0.410000,0.450000,-0.244999,-3.020617,2.000000,69.500000,0.273170,0.095890,0.0039,268.450012,0.04,-0.634624,0.427500,2.897539e-02
50%,225.000000,11.350000,10.950000,11.100000,0.000000,0.000000,10.000000,449.000000,0.359381,0.441096,0.0039,268.450012,0.04,-0.197510,11.025000,1.060016e+01
75%,300.000000,72.165000,69.300000,69.800000,0.000000,0.000000,62.000000,2153.000000,0.556386,1.112329,0.0039,268.450012,0.04,0.085825,69.550000,6.919841e+01
max,530.000000,266.040000,260.850000,264.500000,70.640000,200.000000,89573.000000,88993.000000,7.632813,2.205479,0.0039,268.450012,0.04,0.649057,262.425000,2.633525e+02
